In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class QwenChatbot:
    def __init__(self, model_name="Qwen/Qwen3-0.6B"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)

    def prompt(self, messages):
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=False,
        )

        inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)
        response_ids = self.model.generate(**inputs, max_new_tokens=2048)[0][len(inputs.input_ids[0]):].tolist()
        response = self.tokenizer.decode(response_ids, skip_special_tokens=True)

        return messages + [{"role": "assistant", "content": response}]


In [ ]:
light_weight_chatbot = QwenChatbot("Qwen/Qwen3-1.7B")
chatbot = QwenChatbot("Qwen/Qwen3-4B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
description = chatbot.prompt([
    {"role": "system", "content": "You're best story teller."},
    {"role": "user", "content": "Give me 2-3 sentence story description"},
    {"role": "assistant", "content": "In the darkest depths of the ocean, Iroh the Diver stumbles upon the ruins of a long-lost civilization and unearths secrets guarded for millennia. Joined by mythical beings and a legendary captain, he must navigate ancient magic, monstrous guardians, and mysterious alliances to uncover the truth of Atlantis."},
    {"role": "user", "content": "Give me another 2-3 sentence story description"},

])[-1]['content'].replace("<think>\n\n</think>\n\n", "")

In [ ]:
def parse_json_from_response(response: str):
  return json.loads(response)

In [ ]:
import re
import json

story_metadata = chatbot.prompt([
  {"role": "system", "content": "You only output valid JSON."},
  {"role": "user", "content": "Output the complete json (ONLY JSON) for interactive story quest with the following description: In the darkest depths of the ocean, Iroh the Diver stumbles upon the ruins of a long-lost civilization and unearths secrets guarded for millennia. Joined by mythical beings and a legendary captain, he must navigate ancient magic, monstrous guardians, and mysterious alliances to uncover the truth of Atlantis."},
  {"role": "assistant", "content": """{
  "title": "Deep under ocean",
  "description": "Story about a deep-sea diver who discovers an ancient civilization and meets interesting creatures.",
  "characters": [
    {
      "name": "Iroh the Diver",
      "role": "Diver",
      "traits": ["brave", "curious", "adventurous"],
      "description": "A skilled diver who is determined to explore the depths of the ocean and uncover its secrets.",
      "isMain": true
    },
    {
      "name": "Captain Nemo",
      "role": "Captain of the Nautilus",
      "traits": ["brave", "adventurous", "intelligent"],
      "description": "A legendary captain who has explored the ocean for years and knows its secrets. Not a normal human anymore.",
      "isMain": false
    },
    {
      "name": "Maria",
      "role": "Former Queen of Atlantis",
      "traits": ["wise", "mysterious", "elegant", "sexy", "passionate"],
      "description": "The former queen of Atlantis who has been transformed into a mermaid. She is wise and mysterious, with a deep connection to the ocean.",
      "isMain": false
    },
    {
      "name": "Monster of the Deep",
      "role": "Guardian of Atlantis",
      "traits": ["fearsome", "ancient", "powerful", "dangerous"],
      "description": "A fearsome creature that guards the entrance to Atlantis. It is ancient and powerful, with a deep connection to the ocean.",
      "isMain": false
    }
  ],
  "worldview": {
    "setting": "The story takes place in the depths of the ocean, where Iroh discovers an ancient civilization and meets interesting creatures.",
    "timePeriod": "Modern day",
    "technologyLevel": "Advanced technology for diving and exploration, but the civilization is ancient.",
    "magicSystem": "The ocean has its own magic, with creatures and phenomena that defy explanation. The civilization has its own ancient magic that is tied to the ocean."
  },
  "themes": [
    "Exploration",
    "Adventure",
    "Discovery",
    "Mystery",
    "Transformation"
  ],
  "first_introduction_scene": {
    "text": "Iroh the Diver is preparing for his dive into the depths of the ocean. He is excited and nervous, knowing that he is about to embark on an adventure of a lifetime.",
    "choice": "Dive into the ocean"
  }
}"""},
  {"role": "user", "content": f"Output the complete json for interactive story quest with the following description: {description}"},
])[-1]['content']

print(f"{story_metadata = }")

story_metadata = parse_json_from_response(story_metadata)

story_metadata = '{\n  "title": "Timeless Lake",\n  "description": "A story about a young girl who discovers a mysterious stone that opens a portal to a world where time flows backward, leading her to uncover the truth about her family\'s disappearance.",\n  "characters": [\n    {\n      "name": "Lira",\n      "role": "Protagonist",\n      "traits": ["curious", "brave", "determined"],\n      "description": "A young girl who stumbles upon a mysterious stone and is transported to a world where time flows backward. She must uncover the truth about her family\'s disappearance.",\n      "isMain": true\n    },\n    {\n      "name": "The Guardian of Time",\n      "role": "Ancient being",\n      "traits": ["wise", "mysterious", "ancient"],\n      "description": "An ancient being who guards the portal to the time-reversed world. It is a powerful entity that holds the secrets of time and the past.",\n      "isMain": false\n    },\n    {\n      "name": "Elias",\n      "role": "Former family membe

In [ ]:
story_metadata

{'title': 'Timeless Lake',
 'description': "A story about a young girl who discovers a mysterious stone that opens a portal to a world where time flows backward, leading her to uncover the truth about her family's disappearance.",
 'characters': [{'name': 'Lira',
   'role': 'Protagonist',
   'traits': ['curious', 'brave', 'determined'],
   'description': "A young girl who stumbles upon a mysterious stone and is transported to a world where time flows backward. She must uncover the truth about her family's disappearance.",
   'isMain': True},
  {'name': 'The Guardian of Time',
   'role': 'Ancient being',
   'traits': ['wise', 'mysterious', 'ancient'],
   'description': 'An ancient being who guards the portal to the time-reversed world. It is a powerful entity that holds the secrets of time and the past.',
   'isMain': False},
  {'name': 'Elias',
   'role': 'Former family member',
   'traits': ['mysterious', 'lost', 'connected to the past'],
   'description': "A former family member who 

In [ ]:
from typing import TypedDict, List


class ChoiceDict:
  text: str


class Scene:
  def __init__(self, text, parent_choice: 'Choice', child_choices: List['Choice']) -> None:
      self.text = text
      self.child_choices = child_choices
      self.parent_choice = parent_choice

  def __str__(self) -> str:
      return json.dumps({
          "text": self.text,
          "choices": [{"text": choice.text for choice in self.child_choices}]
      })

  def __repr__(self) -> str:
      return self.__str__()

  @staticmethod
  def build_scene(text, parent_choice: 'Choice', child_choices: List[ChoiceDict]):
    scene = Scene(text, parent_choice, [])
    for choice in child_choices:
      scene.child_choices.append(Choice(choice["text"], scene))
    return scene



class Choice:
  def __init__(self, text: str, parent_scene: Scene, child_scene: Scene = None) -> None:
     self.text = text
     self.parent_scene = parent_scene
     self.child_scene = child_scene

In [ ]:
from uuid import uuid4


first_introduction_scene = Scene.build_scene(
    text=story_metadata['first_introduction_scene']['text'],
    parent_choice=None,
    child_choices=[{"text": story_metadata['first_introduction_scene']['choice']}]
)

In [ ]:
N_SCENES = 30

choices_probabilities = {
    1: 0.75,
    2: 0.2,
    3: 0.05
}

USER_FIRST_MESSAGE_TEMPLATE = (
  "Generate the first scene (1/{n_scene_total}) with 1 choice!"
)

USER_MESSAGE_TEMPLATE = (
  "Player proceeds with \"{choice}\". "
  "Generate next scene ({n_scene}/{n_scene_total}) with {n_choices} choice(s)!"
)

USER_FINAL_MESSAGE_TEMPLATE = (
  "Player proceeds with \"{choice}\". "
  "Generate final scene ({n_scene}/{n_scene_total}) with 1 choice to end the story!"
)

In [ ]:
import random

def build_story(scene: Scene, n_scene_total: int) -> None:
  messages = []
  curr_scene: Scene = scene
  while True:
    if curr_scene.parent_choice is None:
      messages = [{"role": "assistant", "content": str(curr_scene)}] + messages
      break
    messages = [
      {
        "role": "user",
        "content": USER_MESSAGE_TEMPLATE.format(
          choice=curr_scene.parent_choice.text,
          n_scene="{n_scene}",
          n_scene_total=n_scene_total,
          n_choices=len(curr_scene.child_choices),
        )
      },
      {"role": "assistant", "content": str(curr_scene)},
    ] + messages
    curr_scene = curr_scene.parent_choice.parent_scene

  messages = [
    {"role": "system", "content": (
      "You're best story teller. Given the story metadata:\n"
      f"{json.dumps(story_metadata)}\n\n"
      f"You will create a story with {n_scene_total} scenes.\n"
      "IMPORTANT: you output only valid json of the following format:\n"
      '{"text": "<scene description>", "choices": [{"text": "<choice description>"}, ...]}'
    )},
    {"role": "user", "content": USER_FIRST_MESSAGE_TEMPLATE.format(n_scene_total=n_scene_total)},
  ] + messages

  n_scene = 0
  for message in messages:
    if message["role"] == "user":
      n_scene += 1
      message["content"] = message["content"].format(n_scene=n_scene)

  if n_scene == n_scene_total:
    return

  n_scene = n_scene + 1

  for choice in scene.child_choices:
    n_choices = 1
    if n_scene != n_scene_total:
      n_choices = random.choices(
          list(choices_probabilities.keys()),
          weights=list(choices_probabilities.values()),
      k=1)[0]
    curr_messages = messages + [{
      "role": "user",
      "content": USER_FINAL_MESSAGE_TEMPLATE.format(
        choice=choice.text,
        n_scene=n_scene,
        n_scene_total=n_scene_total
      )
    } if n_scene == n_scene_total else {"role": "user", "content": USER_MESSAGE_TEMPLATE.format(
        n_scene=n_scene,
        n_scene_total=n_scene_total,
        choice=choice.text,
        n_choices=n_choices
    )}]
    while True:
      try:
        scene_string = (chatbot.prompt if n_scene < 3 else light_weight_chatbot.prompt)(curr_messages)[-1]['content']
        print(f"{scene_string = }")
        curr_messages.append({"role": "assistant", "content": scene_string})
        scene_json = parse_json_from_response(scene_string)
        print(f"{n_choices = } | {n_scene = } | {scene_json = }")
        print(f"{curr_messages[1:] = }")
        child_scene = Scene.build_scene(
            text=scene_json["text"],
            parent_choice=choice,
            child_choices=scene_json["choices"]
        )
        choice.child_scene = child_scene
        build_story(child_scene, n_scene_total)
        break
      except json.JSONDecodeError as e:
        print(e)
        curr_messages.append({"role": "user", "content": f"json.JSONDecodeError: {str(e)}\nPlease output a valid json."})


build_story(first_introduction_scene, N_SCENES)

scene_string = '{"text": "Lira steps through the portal and finds herself in a strange world where everything moves in reverse. People are walking backward, and objects are falling upward. She looks around and sees ancient ruins in the distance.", "choices": [{"text": "Investigate the ruins"}]}'
n_choices = 1 | n_scene = 2 | scene_json = {'text': 'Lira steps through the portal and finds herself in a strange world where everything moves in reverse. People are walking backward, and objects are falling upward. She looks around and sees ancient ruins in the distance.', 'choices': [{'text': 'Investigate the ruins'}]}
curr_messages[1:] = [{'role': 'user', 'content': 'Generate the first scene (1/30) with 1 choice!'}, {'role': 'assistant', 'content': '{"text": "Lira is walking by the edge of a forgotten lake when she notices a strange stone on the shore. As she picks it up, the world around her begins to change, and she is transported to a world where time flows backward.", "choices": [{"text"

In [ ]:
import json
import textwrap
from graphviz import Digraph

def wrap(text: str, width: int = 30) -> str:
    """
    Wrap `text` into lines no longer than `width` characters.
    Preserves existing newlines.
    """
    # First split on existing newlines to preserve them
    lines = text.splitlines() or [text]
    wrapped_lines = []
    for line in lines:
        # textwrap.fill will insert '\n' as needed
        wrapped_lines.append(textwrap.fill(line, width=width))
    return "\n".join(wrapped_lines)

def visualize_scene(scene: Scene, graph=None, wrap_width: int = 30):
    """
    Recursively adds scenes and choices to a Graphviz Digraph,
    automatically wrapping labels at `wrap_width`.
    """
    if graph is None:
        graph = Digraph(comment='Story Tree')
        graph.attr('node', shape='box', fontname='Helvetica')

    scene_id = str(id(scene))
    scene_label = wrap(scene.text, width=wrap_width)
    # Use double quotes around label so \n is honored
    graph.node(scene_id, label=scene_label)

    for choice in scene.child_choices:
        choice_id = str(id(choice))
        choice_label = wrap(choice.text, width=wrap_width)
        graph.node(choice_id, label=choice_label, shape='ellipse')

        graph.edge(scene_id, choice_id)

        if choice.child_scene:
            child = choice.child_scene
            child_id = str(id(child))
            child_label = wrap(child.text, width=wrap_width)
            graph.node(child_id, label=child_label)
            graph.edge(choice_id, child_id)
            visualize_scene(child, graph, wrap_width=wrap_width)

    return graph

# Example usage:
# root = Scene.build_scene("You wake up in a forest.", None, [{"text": "Go north"}, {"text": "Go south"}])
# # You’d then need to attach child scenes to the root’s choices before visualizing.
dot = visualize_scene(first_introduction_scene)
dot.render('story_tree', view=True)   # writes story_tree.pdf and opens it

'story_tree.pdf'

In [ ]:
import pickle

# ─── Saving ─────────────────────────────────────
with open('root_scene.pkl', 'wb') as f:
    pickle.dump(first_introduction_scene, f)

# # ─── Loading ────────────────────────────────────
# with open('root_scene.pkl', 'rb') as f:
#     loaded_root: Scene = pickle.load(f)

# # Now you can e.g. visualize or traverse loaded_root exactly as before.
